# Figure 5. Scatterplots of ET observed vs. predicted by models

## load data

In [1]:
workdir = 'D:/Field/NATT/modeling_et_afm_v16d/' # this maybe need change according where you place the directory of modeling_et
# Load data
fdata = read.csv(paste0(workdir, 'output/simulation_results.csv'))
fdata2 = read.csv(paste0(workdir, 'output/simulation_results_DIs.csv'))

In [2]:
# combine predicted et from odels
edata = cbind(fdata[,1:2], # 1 for site and 2 for date
              fe_qcflag = fdata$fe_qcflag,   # latent heat flux quality flag
              ebr       = fdata$ebr,         # energy balance ratio
              et_ob     = fdata$et_ob,       # the observed ET
              et_pml    = fdata$et_pml,      # total ET predicted by PML model
              et_sw     = fdata$et_sw,       # total ET predicted by SW model
              et_ts     = fdata$et_ts,       # total ET predicted by TS model
              et_pml_di = fdata2$et_pml_di,   # total ET predicted by PML-DI model
              et_sw_di  = fdata2$et_sw_di,    # total ET predicted by SW-DI model
              et_ts_di  = fdata2$et_ts_di)    # total ET predicted by TS-DI model              
# remove NA records
edata = na.omit(edata)
# remove gap-filling percent >0.2
edata = subset(edata, fe_qcflag<0.2 & ebr > 0.7 & ebr < 1.3)

In [3]:
# 数据质量控制
edata$year = as.numeric(strftime(as.Date(edata$date), format='%Y'))
edata = edata[edata$site != 'stp' | edata$year !=2009,]
edata = edata[edata$site != 'stp' | edata$year !=2015,]

In [4]:
# remove data in AU-DaP and AU-Stp for TS model since there are not three layers
edata[edata$site == 'dap' , ]$et_ts = NA
edata[edata$site == 'stp' , ]$et_ts = NA
edata[edata$site == 'dap' , ]$et_ts_di = NA
edata[edata$site == 'stp' , ]$et_ts_di = NA

In [5]:
# season groups
edata$month <- as.numeric(strftime(as.Date(edata$date),"%m"))
edata$season <- NA
seasons = c('Wet season', 'Brown-down period', 'Dry season', 'Green-up period')

edata[edata$month==12| edata$month<=3, ]$season<- seasons[1]
edata[edata$month==4, ]$season<- seasons[2]
edata[edata$month>=5  & edata$month<=10, ]$season <- seasons[3]
edata[edata$month==11, ]$season <- seasons[4]

name_cols = c('site','season', 'et_ob', 'et_pr', 'modeltype')
# pml
pdata1 <- data.frame(edata$site, edata$season, 
                    edata$et_ob, edata$et_pml, rep('PML', length(edata$et_ob)))
colnames(pdata1) = name_cols
# sw
pdata2 <- data.frame(edata$site, edata$season,
                     edata$et_ob, edata$et_sw, rep('SW', length(edata$et_ob)))
colnames(pdata2) = name_cols
# ts
pdata3 <- data.frame(edata$site, edata$season, 
                    edata$et_ob, edata$et_ts, rep('TS', length(edata$et_ob)))
colnames(pdata3) = name_cols
# PML-DI
pdata4 <- data.frame(edata$site, edata$season, 
                    edata$et_ob, edata$et_pml_di, rep('PML-DI', length(edata$et_ob)))
colnames(pdata4) = name_cols
# SW-DI
pdata5 <- data.frame(edata$site, edata$season, 
                    edata$et_ob, edata$et_sw_di, rep('SW-DI', length(edata$et_ob)))
colnames(pdata5) = name_cols
# TS-DI
pdata6 <- data.frame(edata$site, edata$season, 
                    edata$et_ob, edata$et_ts_di, rep('TS-DI', length(edata$et_ob)))
colnames(pdata6) = name_cols

pdata <- rbind(pdata1, pdata2, pdata3, pdata4, pdata5, pdata6)
pdata$siten = NA
pdata[pdata$site=='how', ]$siten = 'AU-How'
pdata[pdata$site=='das', ]$siten = 'AU-DaS'
pdata[pdata$site=='dry', ]$siten = 'AU-Dry'
pdata[pdata$site=='asm', ]$siten = 'AU-ASM'
pdata[pdata$site=='dap', ]$siten = 'AU-DaP'
pdata[pdata$site=='stp', ]$siten = 'AU-StP'

In [6]:
pdata$siten <- factor(pdata$siten, level=c('AU-How', 'AU-DaS','AU-Dry', 'AU-ASM', 'AU-DaP','AU-StP'),
                    labels=c('AU-How', 'AU-DaS','AU-Dry', 'AU-ASM', 'AU-DaP','AU-StP'))
pdata$season <- factor(pdata$season, level=seasons, labels=seasons)
modeltypes = c('SW', 'SW-DI', 'PML', 'PML-DI', 'TS', 'TS-DI')
pdata$modeltype = factor(pdata$modeltype, level=modeltypes, labels=modeltypes)
# store data for ploting figure 
# write.csv(pdata, file = 'fig5_pdata.csv')

In [7]:
col_season = c('#339900', #'Wet season'
               '#996600',  #'Brown-down period'
               '#CCCC33',  #'Dry season'
               '#FF0033')  #'Green-up period'         

In [8]:
#   plot
library(ggplot2)

et_scatter <- ggplot(aes(x = et_ob , y = et_pr), data = pdata)+ 
  # facet_grid(modeltype ~ siten) +
  facet_grid(siten ~ modeltype ) +
  geom_abline(slope=1,lty=2, intercept=0, col='grey') + 
  geom_point(size=0.1,shape=1,aes(col=season)) +
  geom_smooth(method=lm, lwd = 0.8, col='gray2') + 
  coord_fixed() +
  theme_bw() +
  theme(panel.grid.major = element_blank(),
    panel.grid.minor = element_blank()) +
  scale_color_manual(values=col_season) + 
  scale_x_continuous(limit=c(0, 7)) + scale_y_continuous(limit=c(0, 7))

Warning message:
"package 'ggplot2' was built under R version 3.5.3"

In [9]:
et_scatter2 <- et_scatter +
  xlab(expression(paste('Observed ET (mm d'^'-1',')')))+
  ylab(expression(paste('Predicted  ET (mm d'^'-1',')'))) +
  theme(legend.position = "bottom") +
  labs(color=" ")

In [10]:
pdf("Fig.7_scatter_obs_predict_6models.pdf",width=9, height=9)
print(et_scatter2)
dev.off()

Warning message:
"Removed 6457 rows containing non-finite values (stat_smooth)."Warning message:
"Removed 6457 rows containing missing values (geom_point)."

png 
  2

In [11]:
postscript("Fig.7_scatter_obs_predict_6models.eps",width=7, height=9)
print(et_scatter2)
dev.off()

Warning message:
"Removed 6457 rows containing non-finite values (stat_smooth)."Warning message:
"Removed 6457 rows containing missing values (geom_point)."Warning message in grid.Call.graphics(C_polygon, x$x, x$y, index):
"此装置不支持半透明：每一页将被报告一次"

png 
  2